<a href="https://colab.research.google.com/github/ferran9908/MilkBasketOptimization/blob/master/MilkBasket_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install pandasql
import numpy as np
import pandas as pd
from IPython.display import display
import pandasql
import matplotlib.pyplot as plt



In [0]:
!ls "/content/drive/My Drive/MilkBasket"

In [0]:
data = pd.read_csv("/content/drive/My Drive/MilkBasket/hackathon_data.csv")
#data = data.head(2000000)

In [0]:
pd.plotting.scatter_matrix(TrainData, alpha = 0.3, figsize = (14,10), diagonal='kde');
plt.show()

In [0]:
from sklearn.model_selection import train_test_split
TrainData,TestData = train_test_split(data,test_size=0.66)

In [0]:
CVdata,TestD = train_test_split(TestData,test_size=0.5)

In [0]:
TestD.head()

##Convert Dates to days and time to discrete hours

In [0]:
import datetime
def day_time(data):
  day=[]
  time=[]
  date=data['product_addedtobasket_on']
  dates=date.tolist()
  for date in dates:
    a,b=date.split()
    a=datetime.datetime.strptime(a,'%Y-%m-%d').strftime('%A')
    b=int(b.split(':')[0])
    day.append(a)
    time.append(b)
  return day,time
  

In [0]:
def day(data):
  day=[]
  date=data['order_date']
  dates=date.tolist()
  for date in dates:
    a = date
    #a,b=date.split()
    a=datetime.datetime.strptime(a,'%Y-%m-%d').strftime('%A')
    #b=int(b.split(':')[0])
    day.append(a)
    #time.append(b)
  return day

In [0]:
a,b = day_time(TrainData)
d = day(TrainData)

In [0]:
TrainData['day'] = a
TrainData['time'] = b
TrainData['order_date'] = d

In [0]:
a,b = day_time(TestD)
d = day(TestD)

TestD['day'] = a
TestD['time'] = b
TestD['order_date'] = d

##Classification of Days and Hours into Weekdays and Weekends

In [0]:
a,b = day_time(CVdata)
d = day(CVdata)

CVdata['day'] = a
CVdata['time'] = b
CVdata['order_date'] = d


In [0]:
def weekend_weekday(x):
  if(x!='Saturday' or x!='Sunday'):
    return 0
  else:
    return 1
  
TrainData['day'] = TrainData['day'].apply(weekend_weekday)
TrainData['order_date'] = TrainData['order_date'].apply(weekend_weekday)

TestD['day'] = TestD['day'].apply(weekend_weekday)
TestD['order_date'] = TestD['order_date'].apply(weekend_weekday)

In [0]:
CVdata['day'] = CVdata['day'].apply(weekend_weekday)
CVdata['order_date'] = CVdata['order_date'].apply(weekend_weekday)

##Finding Outliers to Identify Unique Customer Behaviour

In [0]:
TrainData = TrainData.drop(columns = ['product_addedtobasket_on'])
TestD = TestD.drop(columns = ['product_addedtobasket_on'])

CVdata = CVdata.drop(columns = ['product_addedtobasket_on'])


In [0]:
def total_cost_outlier(x):
  if(x>10000):
    return 10000
  else:
    return x
    

In [0]:
TrainData['total_cost'] = TrainData['total_cost'].apply(total_cost_outlier)
TestD['total_cost'] = TestD['total_cost'].apply(total_cost_outlier)

CVdata['total_cost'] = CVdata['total_cost'].apply(total_cost_outlier)

In [0]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(TrainData)
standardized_data = ss.transform(TrainData)
test = ss.transform(TestD)
cv = ss.transform(CVdata)

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=8)
pca.fit(standardized_data)



In [0]:
standardized_data = pca.transform(standardized_data)
test = pca.transform(test)
test.shape

In [0]:
from sklearn.cluster import KMeans

In [0]:
clf = KMeans(n_clusters=8)
clf.fit(standardized_data)

In [0]:
test = test.drop(columns=8)

In [0]:
pred = clf.predict(test)
print(pred)
test = pd.DataFrame(test)


In [0]:
# Nice Pythonic way to get the indices of the points for each corresponding cluster
mydict = []
mydict = {i: np.where(clf.labels_ == i)[0] for i in range(clf.n_clusters)}
print (mydict)


#Transform this dictionary into list (if you need a list as result)
dictlist = []
for key, value in mydict.items():
    temp = [key,value]
    dictlist.append(temp)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(test,pred)

In [0]:
# test_data = pd.DataFrame(test,columns = ['Dimension1','Dimension2'])
# def biplot(data, reduced_data, pca):
    
#     fig, ax = plt.subplots(figsize = (14,8))
    
#     # scatterplot of the reduced data 
#     ax.scatter(x=reduced_data.loc[:, 'Dimension1'], y=reduced_data.loc[:, 'Dimension2'], facecolors='b', edgecolors='b', s=70, alpha=0.5)
    
#     feature_vectors = pca.components_.T

#     # using scaling factors to make the arrows
#     arrow_size, text_pos = 7.0, 8.0,

#     # projections of the original features
#     for i, v in enumerate(feature_vectors):
#         ax.arrow(0, 0, arrow_size*v[0], arrow_size*v[1], head_width=0.2, head_length=0.2, linewidth=2, color='red')
#         #ax.text(v[0]*text_pos, v[1]*text_pos, TestD.columns[i], color='black', ha='center', va='center', fontsize=18)

#     ax.set_xlabel("Dimension 1", fontsize=14)
#     ax.set_ylabel("Dimension 2", fontsize=14)
#     ax.set_title("PC plane with original feature projections.", fontsize=16);
#     return ax

# biplot(TestD, test_data, pca)


In [0]:
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [0]:
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score

# range_n_clusters = [2, 3, 4, 6, 10]            # clusters range you want to select
# #dataToFit = [[12,23],[112,46],[45,23]]  # sample data
# best_clusters = 0                       # best cluster number which you will get
# previous_silh_avg = 0.0

# for n_clusters in range_n_clusters:
#     clusterer = KMeans(n_clusters=n_clusters)
#     cluster_labels = clusterer.fit_predict(TrainData)
#     silhouette_avg = silhouette_score(TrainData, cluster_labels)
#     if silhouette_avg > previous_silh_avg:
#         previous_silh_avg = silhouette_avg
#         best_clusters = n_clusters
#     print(best_clusters)